In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import ot

ModuleNotFoundError: No module named 'ot'

In [7]:
# Read the CSV file for cambridge
df_user_ca = pd.read_csv('/Users/naroacorettisanchez/Documents/GitHub/MultifunctionalVehicleFleets/includes/Demand/user_demand_cambridge_oct7_2019_week.csv')
df_food_ca= pd.read_csv('/Users/naroacorettisanchez/Documents/GitHub/MultifunctionalVehicleFleets/includes/Demand/food_demand_cambridge_week.csv')

#Read the CSV file for San Sebastian
df_user_ss = pd.read_csv('/Users/naroacorettisanchez/Documents/GitHub/DataSS/Rides/ride_demand_ss_1week_scattered.csv')
df_food_ss = pd.read_csv('/Users/naroacorettisanchez/Documents/GitHub/DataSS/Deliveries/delivery_demand_ss_1week_scattered.csv')

In [37]:


def count_points_in_squares(df, lat_col_start, lon_col_start, lat_col_end, lon_col_end, lat_bins, lon_bins):
    # Create bins for latitude and longitude for both start and end points
    df['start_lat_bin'] = pd.cut(df[lat_col_start], bins=lat_bins)
    df['start_lon_bin'] = pd.cut(df[lon_col_start], bins=lon_bins)
    df['end_lat_bin'] = pd.cut(df[lat_col_end], bins=lat_bins)
    df['end_lon_bin'] = pd.cut(df[lon_col_end], bins=lon_bins)

    # Count start points within each square
    start_counts = df.groupby(['start_lat_bin', 'start_lon_bin']).size().reset_index(name='start_point_count')

    # Count end points within each square
    end_counts = df.groupby(['end_lat_bin', 'end_lon_bin']).size().reset_index(name='end_point_count')

    # Pivot the tables to get counts in matrix form for start and end points
    df_matrix_start = start_counts.pivot_table(index='start_lat_bin', columns='start_lon_bin', values='start_point_count', fill_value=0)
    df_matrix_end = end_counts.pivot_table(index='end_lat_bin', columns='end_lon_bin', values='end_point_count', fill_value=0)

    return df_matrix_start, df_matrix_end

# Specify the space in latitude and longitude and the number of bins
lat_bins = np.linspace(start=min(df_user_ca['start_lat'].min(), df_user_ca['target_lat'].min()),
                       stop=max(df_user_ca['start_lat'].max(), df_user_ca['target_lat'].max()),
                       num=10)

lon_bins = np.linspace(start=min(df_user_ca['start_lon'].min(), df_user_ca['target_lon'].min()),
                       stop=max(df_user_ca['start_lon'].max(), df_user_ca['target_lon'].max()),
                       num=10)

# Count points within squares for df_user_ca
df_matrix_start_user_ca, df_matrix_end_user_ca = count_points_in_squares(df_user_ca, 'start_lat', 'start_lon', 'target_lat', 'target_lon', lat_bins, lon_bins)

# Repeat the same process for other DataFrames (df_food_ca, df_user_ss, df_food_ss)
df_matrix_start_food_ca, df_matrix_end_food_ca = count_points_in_squares(df_food_ca, 'start_latitude', 'start_longitude', 'end_latitude', 'end_longitude', lat_bins, lon_bins)



In [65]:
df_matrix_start_user_ca_sum = np.sum(np.abs(df_matrix_start_user_ca.values))
df_matrix_start_food_ca_sum = np.sum(np.abs(df_matrix_start_food_ca.values))

In [39]:
ca_start = df_matrix_start_user_ca - df_matrix_start_food_ca
ca_start


start_lon_bin,"(-71.156, -71.146]","(-71.146, -71.136]","(-71.136, -71.126]","(-71.126, -71.116]","(-71.116, -71.106]","(-71.106, -71.097]","(-71.097, -71.087]","(-71.087, -71.077]","(-71.077, -71.067]"
start_lat_bin,,,,,,,,,
"(42.353, 42.359]",0,0,0,0,276,707,308,0,0
"(42.359, 42.364]",0,0,0,0,162,57,-1027,-478,0
"(42.364, 42.369]",0,0,0,0,35,-3096,-79,-320,-542
"(42.369, 42.374]",0,2,1,-5877,-1101,-16,-390,-966,-129
"(42.374, 42.38]",-229,7,99,317,-26,9,0,0,0
"(42.38, 42.385]",7,35,-74,-942,29,0,0,0,0
"(42.385, 42.39]",0,5,35,-1416,1,0,0,0,0
"(42.39, 42.395]",12,51,-98,-833,0,0,0,0,0
"(42.395, 42.4]",35,25,-443,0,0,0,0,0,0


In [66]:
ca_start_sum = np.sum(np.abs(ca_start.values))
ca_start_sum / (df_matrix_start_user_ca_sum + df_matrix_start_food_ca_sum)

0.45826465873428013

In [64]:
ca_end = df_matrix_end_user_ca - df_matrix_end_food_ca
ca_end

end_lon_bin,"(-71.156, -71.146]","(-71.146, -71.136]","(-71.136, -71.126]","(-71.126, -71.116]","(-71.116, -71.106]","(-71.106, -71.097]","(-71.097, -71.087]","(-71.087, -71.077]","(-71.077, -71.067]"
end_lat_bin,,,,,,,,,
"(42.353, 42.359]",0.000000,0.000000,0.000000,0.000000,-0.007067,0.003567,-0.001897,0.000000,0.000000
"(42.359, 42.364]",0.000000,0.000000,-0.000113,0.000000,-0.020117,-0.015308,0.019485,0.009257,0.000000
"(42.364, 42.369]",0.000000,0.000000,0.000000,-0.000158,-0.010612,-0.014563,-0.000045,-0.017204,-0.009596
"(42.369, 42.374]",0.000000,-0.000497,-0.002709,-0.030096,-0.056287,-0.014947,-0.014698,-0.027974,-0.008331
"(42.374, 42.38]",-0.008354,-0.002845,-0.001671,-0.009776,-0.013592,-0.000181,0.000000,0.000000,0.000000
"(42.38, 42.385]",-0.000452,-0.004177,-0.011402,-0.024903,-0.006593,0.000000,0.000000,0.000000,0.000000
"(42.385, 42.39]",-0.000835,-0.000700,-0.002032,-0.020478,0.000045,0.000000,0.000000,0.000000,0.000000
"(42.39, 42.395]",0.000113,0.001197,-0.007135,-0.013321,0.000000,0.000000,0.000000,0.000000,0.000000
"(42.395, 42.4]",0.000813,-0.000248,-0.009934,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [68]:
ca_end_sum = np.sum(np.abs(ca_end.values))
ca_end_sum / (df_matrix_start_user_ca_sum + df_matrix_start_food_ca_sum)

0.42532342913910276

In [48]:
ca_diff_1= df_matrix_end_user_ca - df_matrix_start_food_ca
ca_diff_1

end_lon_bin,"(-71.156, -71.146]","(-71.146, -71.136]","(-71.136, -71.126]","(-71.126, -71.116]","(-71.116, -71.106]","(-71.106, -71.097]","(-71.097, -71.087]","(-71.087, -71.077]","(-71.077, -71.067]"
end_lat_bin,,,,,,,,,
"(42.353, 42.359]",0,0,0,0,333,713,303,0,0
"(42.359, 42.364]",0,0,0,0,174,-76,-1049,-409,0
"(42.364, 42.369]",0,0,0,0,56,-3038,-29,-319,-539
"(42.369, 42.374]",0,0,0,-5860,-1094,-125,-408,-1015,-117
"(42.374, 42.38]",-217,7,108,308,-20,11,0,0,0
"(42.38, 42.385]",5,38,-64,-929,24,0,0,0,0
"(42.385, 42.39]",0,2,22,-1421,2,0,0,0,0
"(42.39, 42.395]",5,53,-103,-841,0,0,0,0,0
"(42.395, 42.4]",36,42,-441,0,0,0,0,0,0


In [69]:
ca_diff_1_sum = np.sum(np.abs(ca_diff_1.values))
ca_diff_1_sum / (df_matrix_start_user_ca_sum + df_matrix_start_food_ca_sum)

0.45959675780632636

In [49]:
ca_diff_2= df_matrix_start_user_ca - df_matrix_end_food_ca
ca_diff_2

start_lon_bin,"(-71.156, -71.146]","(-71.146, -71.136]","(-71.136, -71.126]","(-71.126, -71.116]","(-71.116, -71.106]","(-71.106, -71.097]","(-71.097, -71.087]","(-71.087, -71.077]","(-71.077, -71.067]"
start_lat_bin,,,,,,,,,
"(42.353, 42.359]",0,0,0,0,-370,152,-79,0,0
"(42.359, 42.364]",0,0,-5,0,-903,-545,885,341,0
"(42.364, 42.369]",0,0,0,-7,-491,-703,-52,-763,-428
"(42.369, 42.374]",0,-20,-119,-1350,-2500,-553,-633,-1190,-381
"(42.374, 42.38]",-382,-126,-83,-424,-608,-10,0,0,0
"(42.38, 42.385]",-18,-188,-515,-1116,-287,0,0,0,0
"(42.385, 42.39]",-37,-28,-77,-902,1,0,0,0,0
"(42.39, 42.395]",12,51,-311,-582,0,0,0,0,0
"(42.395, 42.4]",35,-28,-442,0,0,0,0,0,0


In [70]:
ca_diff_2_sum = np.sum(np.abs(ca_diff_2.values))
ca_diff_2_sum / (df_matrix_start_user_ca_sum + df_matrix_start_food_ca_sum)

0.4229527443498679

In [41]:
# Specify the space in latitude and longitude and the number of bins
lat_bins = np.linspace(start=min(df_user_ss['start_lat'].min(), df_user_ss['target_lat'].min()),
                       stop=max(df_user_ss['start_lat'].max(), df_user_ss['target_lat'].max()),
                       num=10)

lon_bins = np.linspace(start=min(df_user_ss['start_lon'].min(), df_user_ss['target_lon'].min()),
                       stop=max(df_user_ss['start_lon'].max(), df_user_ss['target_lon'].max()),
                       num=10)

# Count points within squares for df_user_ca
df_matrix_start_user_ss, df_matrix_end_user_ss = count_points_in_squares(df_user_ss, 'start_lat', 'start_lon', 'target_lat', 'target_lon', lat_bins, lon_bins)

# Repeat the same process for other DataFrames (df_food_ca, df_user_ss, df_food_ss)
df_matrix_start_food_ss, df_matrix_end_food_ss = count_points_in_squares(df_food_ss, 'start_latitude', 'start_longitude', 'end_latitude', 'end_longitude', lat_bins, lon_bins)

In [80]:
df_matrix_start_user_ss_sum = np.sum(np.abs(df_matrix_start_user_ss.values))
print(df_matrix_start_user_ss_sum)
df_matrix_start_food_ss_sum = np.sum(np.abs(df_matrix_start_food_ss.values))
print(df_matrix_start_food_ss_sum)

19296
6499


In [72]:
ss_start = df_matrix_start_user_ss - df_matrix_start_food_ss
ss_start


start_lon_bin,"(-2.021, -2.011]","(-2.011, -2.001]","(-2.001, -1.99]","(-1.99, -1.98]","(-1.98, -1.97]","(-1.97, -1.96]","(-1.96, -1.949]","(-1.949, -1.939]","(-1.939, -1.929]"
start_lat_bin,,,,,,,,,
"(43.29, 43.294]",0,3,4,0,0,0,42,0,0
"(43.294, 43.299]",0,17,83,33,0,2,101,0,0
"(43.299, 43.303]",114,463,284,529,556,27,56,0,0
"(43.303, 43.307]",321,267,303,520,495,229,53,1,0
"(43.307, 43.312]",321,235,262,458,647,464,62,-1540,0
"(43.312, 43.316]",90,904,206,1144,420,208,167,11,12
"(43.316, 43.32]",6,92,0,167,1016,460,231,52,69
"(43.32, 43.325]",0,1,55,275,538,509,126,17,-70
"(43.325, 43.329]",0,0,5,157,118,395,4,0,0


In [73]:
ss_start_sum = np.sum(np.abs(ss_start.values))
ss_start_sum / (df_matrix_start_user_ss_sum + df_matrix_start_food_ss_sum)



0.620934289591006

In [81]:
df_matrix_end_user_ss

end_lon_bin,"(-2.021, -2.011]","(-2.011, -2.001]","(-2.001, -1.99]","(-1.99, -1.98]","(-1.98, -1.97]","(-1.97, -1.96]","(-1.96, -1.949]","(-1.949, -1.939]","(-1.939, -1.929]"
end_lat_bin,,,,,,,,,
"(43.29, 43.294]",0,1,3,0,0,1,57,0,0
"(43.294, 43.299]",0,22,35,15,0,4,116,0,0
"(43.299, 43.303]",111,538,193,424,557,39,48,0,0
"(43.303, 43.307]",301,363,239,459,800,239,63,0,0
"(43.307, 43.312]",364,650,281,463,719,501,52,4,0
"(43.312, 43.316]",100,982,184,1978,393,169,114,6,10
"(43.316, 43.32]",5,91,0,673,1449,483,225,44,67
"(43.32, 43.325]",0,1,49,1608,1426,534,103,19,2
"(43.325, 43.329]",0,0,9,162,304,441,4,0,0


In [82]:
df_matrix_end_food_ss

end_lon_bin,"(-2.021, -2.011]","(-2.011, -2.001]","(-2.001, -1.99]","(-1.99, -1.98]","(-1.98, -1.97]","(-1.97, -1.96]","(-1.96, -1.949]","(-1.949, -1.939]","(-1.939, -1.929]"
end_lat_bin,,,,,,,,,
"(43.29, 43.294]",0,5,28,17,11,16,27,0,0
"(43.294, 43.299]",6,23,17,6,3,3,41,4,0
"(43.299, 43.303]",49,37,63,54,78,8,23,9,0
"(43.303, 43.307]",47,76,123,137,330,70,39,0,0
"(43.307, 43.312]",33,131,143,241,157,103,28,3,0
"(43.312, 43.316]",13,250,113,412,112,12,121,50,135
"(43.316, 43.32]",0,24,0,331,264,168,198,137,238
"(43.32, 43.325]",0,3,1,291,472,190,43,169,179
"(43.325, 43.329]",0,0,0,25,110,28,0,1,84


In [43]:
ss_end = df_matrix_end_user_ss - df_matrix_end_food_ss
ss_end

end_lon_bin,"(-2.021, -2.011]","(-2.011, -2.001]","(-2.001, -1.99]","(-1.99, -1.98]","(-1.98, -1.97]","(-1.97, -1.96]","(-1.96, -1.949]","(-1.949, -1.939]","(-1.939, -1.929]"
end_lat_bin,,,,,,,,,
"(43.29, 43.294]",0,-4,-25,-17,-11,-15,30,0,0
"(43.294, 43.299]",-6,-1,18,9,-3,1,75,-4,0
"(43.299, 43.303]",62,501,130,370,479,31,25,-9,0
"(43.303, 43.307]",254,287,116,322,470,169,24,0,0
"(43.307, 43.312]",331,519,138,222,562,398,24,1,0
"(43.312, 43.316]",87,732,71,1566,281,157,-7,-44,-125
"(43.316, 43.32]",5,67,0,342,1185,315,27,-93,-171
"(43.32, 43.325]",0,-2,48,1317,954,344,60,-150,-177
"(43.325, 43.329]",0,0,9,137,194,413,4,-1,-84


In [74]:
ss_end_sum = np.sum(np.abs(ss_end.values))
ss_end_sum / (df_matrix_start_user_ss_sum + df_matrix_start_food_ss_sum)

0.5749951540996318

In [47]:
ss_diff_1= df_matrix_end_user_ss - df_matrix_start_food_ss
ss_diff_1 

end_lon_bin,"(-2.021, -2.011]","(-2.011, -2.001]","(-2.001, -1.99]","(-1.99, -1.98]","(-1.98, -1.97]","(-1.97, -1.96]","(-1.96, -1.949]","(-1.949, -1.939]","(-1.939, -1.929]"
end_lat_bin,,,,,,,,,
"(43.29, 43.294]",0,1,3,0,0,1,57,0,0
"(43.294, 43.299]",0,22,35,15,0,4,116,0,0
"(43.299, 43.303]",111,503,193,424,557,39,48,0,0
"(43.303, 43.307]",301,359,239,459,545,239,63,0,0
"(43.307, 43.312]",361,332,281,463,692,501,52,-1537,0
"(43.312, 43.316]",100,854,184,1213,376,169,114,6,10
"(43.316, 43.32]",5,91,0,144,1007,447,225,44,67
"(43.32, 43.325]",0,1,49,359,587,505,103,13,-68
"(43.325, 43.329]",0,0,9,162,98,441,4,0,0


In [75]:
ss_diff_1_sum = np.sum(np.abs(ss_diff_1.values))
ss_diff_1_sum / (df_matrix_start_user_ss_sum + df_matrix_start_food_ss_sum)

0.6205853847644892

In [77]:
ss_diff_2= df_matrix_start_user_ss - df_matrix_end_food_ss
ss_diff_2

start_lon_bin,"(-2.021, -2.011]","(-2.011, -2.001]","(-2.001, -1.99]","(-1.99, -1.98]","(-1.98, -1.97]","(-1.97, -1.96]","(-1.96, -1.949]","(-1.949, -1.939]","(-1.939, -1.929]"
start_lat_bin,,,,,,,,,
"(43.29, 43.294]",0,-2,-24,-17,-11,-16,15,0,0
"(43.294, 43.299]",-6,-6,66,27,-3,-1,60,-4,0
"(43.299, 43.303]",65,461,221,475,478,19,33,-9,0
"(43.303, 43.307]",274,195,180,383,420,159,14,1,0
"(43.307, 43.312]",291,422,119,217,517,361,34,-2,0
"(43.312, 43.316]",77,782,93,1497,325,196,46,-39,-123
"(43.316, 43.32]",6,68,0,365,1194,328,33,-85,-169
"(43.32, 43.325]",0,-2,54,1233,905,348,83,-146,-179
"(43.325, 43.329]",0,0,5,132,214,367,4,-1,-84


In [78]:
ss_diff_2_sum = np.sum(np.abs(ss_diff_2.values))
ss_diff_2_sum / (df_matrix_start_user_ss_sum + df_matrix_start_food_ss_sum)

0.5734056987788331